
# K-Means Song Mood Clustering — EDA → Kesimpulan
Notebook ini mengklaster **lagu** berdasarkan **fitur audio** (valence, energy, danceability, acousticness, instrumentalness, dll.) menggunakan **K-Means**.
Struktur:
1. Config & Load Data  
2. EDA (missingness, histogram, korelasi)  
3. Feature Engineering (log tempo, duration menit, encoding key/mode, loudness z)  
4. Scaling & (opsional) PCA  
5. Pemilihan k: Elbow + Silhouette + Calinski–Harabasz + Davies–Bouldin  
6. K-Means/Minibatch final (auto-pick)  
7. Profiling klaster (mean/std fitur, representative tracks)  
8. Visualisasi: PCA 2D scatter, ukuran klaster, kurva metrik vs k  
9. Validasi eksternal: Purity & NMI vs genre (jika tersedia)  
10. Stability bootstrap (ARI)  
11. Ablasi: RAW vs PCA  
12. Kesimpulan


In [ ]:

from pathlib import Path
import pandas as pd
import numpy as np

DATA_PATH = Path("/mnt/data/your_spotify_audio_features.csv")  # Ganti path

COLS_CANDIDATES = {
    "track_id":   ["track_id","id","uri","track_uri"],
    "track_name": ["track_name","name","track","song","title"],
    "artist":     ["artist","artists","artist_name","artist_names"],
    "genre":      ["genre","playlist_genre","tag","primary_genre"],
    "valence":    ["valence"],
    "energy":     ["energy"],
    "danceability": ["danceability"],
    "acousticness": ["acousticness"],
    "instrumentalness": ["instrumentalness"],
    "liveness":   ["liveness"],
    "speechiness":["speechiness"],
    "tempo":      ["tempo","bpm"],
    "loudness":   ["loudness"],
    "duration_ms":["duration_ms","duration"],
    "time_signature": ["time_signature","timesig"],
    "key":        ["key"],
    "mode":       ["mode"]
}

SAMPLE_EDA = 100000
K_RANGE = list(range(6, 15))
USE_PCA_FOR_CLUSTER = False
N_PCA_COMPONENTS = 20
MINIBATCH_THRESHOLD = 300_000
MINIBATCH_BATCH_SIZE = 4096
BOOTSTRAP_RUNS = 20
BOOTSTRAP_FRAC = 0.8
RANDOM_STATE = 42
PRINT_ROWS = 10
print("Config loaded.")


In [ ]:

def autodetect_columns(df, candidates):
    mapping = {}
    cols_lower = {c.lower(): c for c in df.columns}
    for key, cand_list in candidates.items():
        found = None
        for cand in cand_list:
            if cand.lower() in cols_lower:
                found = cols_lower[cand.lower()]
                break
        mapping[key] = found
    return mapping

def read_data(path: Path, nrows=None):
    return pd.read_csv(path, nrows=nrows)

df_head = read_data(DATA_PATH, nrows=5000)
colmap = autodetect_columns(df_head, COLS_CANDIDATES)
print("Column mapping (auto):", colmap)

df = read_data(DATA_PATH, nrows=None)
print("Shape:", df.shape)
display(df.head(PRINT_ROWS))


In [ ]:

# EDA
missing = df.isna().sum().sort_values(ascending=False)
display(missing.head(30))

num_keys = ["valence","energy","danceability","acousticness","instrumentalness",
            "liveness","speechiness","tempo","loudness","duration_ms","time_signature","key","mode"]
num_cols = [colmap[k] for k in num_keys if colmap.get(k) is not None]
num_cols = [c for c in num_cols if c in df.columns]

if len(df) > SAMPLE_EDA:
    df_eda = df.sample(SAMPLE_EDA, random_state=RANDOM_STATE)
else:
    df_eda = df.copy()

print("Numeric columns used for EDA:", num_cols)

import matplotlib.pyplot as plt

for c in num_cols[:8]:
    plt.figure()
    df_eda[c].dropna().astype(float).hist(bins=50)
    plt.title(f"Histogram: {c}")
    plt.xlabel(c); plt.ylabel("Count")
    plt.tight_layout()
    plt.show()

if len(num_cols) >= 2:
    corr = df_eda[num_cols].corr(numeric_only=True)
    plt.figure(figsize=(8,6))
    im = plt.imshow(corr.values)
    plt.xticks(range(len(num_cols)), num_cols, rotation=90)
    plt.yticks(range(len(num_cols)), num_cols)
    plt.title("Correlation (Pearson)")
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.show()


In [ ]:

# Feature Engineering
work = df.copy()

if colmap.get("duration_ms") and colmap["duration_ms"] in work.columns:
    work["duration_min"] = work[colmap["duration_ms"]] / 60000.0
else:
    work["duration_min"] = np.nan

if colmap.get("tempo") and colmap["tempo"] in work.columns:
    work["log_tempo"] = np.log1p(work[colmap["tempo"]])
else:
    work["log_tempo"] = np.nan

if colmap.get("key") and colmap["key"] in work.columns:
    key_raw = work[colmap["key"]].astype(float)
    work["key_sin"] = np.sin(2*np.pi*key_raw/12.0)
    work["key_cos"] = np.cos(2*np.pi*key_raw/12.0)
else:
    work["key_sin"] = np.nan
    work["key_cos"] = np.nan

if colmap.get("mode") and colmap["mode"] in work.columns:
    work["mode_bin"] = work[colmap["mode"]].astype(float)
else:
    work["mode_bin"] = np.nan

if colmap.get("loudness") and colmap["loudness"] in work.columns:
    loud = work[colmap["loudness"]].astype(float)
    work["loudness_z"] = (loud - loud.mean()) / (loud.std(ddof=0) + 1e-9)
else:
    work["loudness_z"] = np.nan

feature_list = []
for k in ["valence","energy","danceability","acousticness","instrumentalness","liveness","speechiness"]:
    if colmap.get(k):
        feature_list.append(colmap[k])

for extra in ["log_tempo","loudness_z","duration_min","key_sin","key_cos","mode_bin","time_signature"]:
    if (extra in work.columns) or (colmap.get(extra) and colmap[extra] in work.columns):
        feature_list.append(extra if extra in work.columns else colmap[extra])

X = work[feature_list].copy()
pre_rows = len(X)
X = X.dropna()
print(f"Dropped rows due to NaN in features: {pre_rows - len(X)}")

id_col   = colmap.get("track_id")
name_col = colmap.get("track_name")
artist_col = colmap.get("artist")
meta_cols = [c for c in [id_col, name_col, artist_col, colmap.get("genre")] if c]
meta = work.loc[X.index, meta_cols] if meta_cols else None

print("Feature matrix shape:", X.shape)
print("Feature columns:", list(X.columns))


In [ ]:

# Scaling & PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.values)

USE_PCA_FOR_CLUSTER = False
N_PCA_COMPONENTS = 20
if USE_PCA_FOR_CLUSTER:
    pca = PCA(n_components=N_PCA_COMPONENTS, random_state=42)
    X_model = pca.fit_transform(X_scaled)
    print("PCA explained variance ratio (cum):", pca.explained_variance_ratio_.sum())
else:
    X_model = X_scaled

print("Model space:", X_model.shape)


In [ ]:

# K selection
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import numpy as np
import pandas as pd

MINIBATCH_THRESHOLD = 300_000
MINIBATCH_BATCH_SIZE = 4096
use_minibatch = (len(X_model) > MINIBATCH_THRESHOLD)
print("Using MiniBatchKMeans:", use_minibatch)

rows = []; inertias = []
for k in range(6, 15):
    if use_minibatch:
        km = MiniBatchKMeans(n_clusters=k, batch_size=MINIBATCH_BATCH_SIZE, n_init=10, random_state=42)
    else:
        km = KMeans(n_clusters=k, n_init=15, random_state=42)
    lab = km.fit_predict(X_model)
    inertias.append(km.inertia_)
    try: sil = silhouette_score(X_model, lab)
    except: sil = np.nan
    try: ch = calinski_harabasz_score(X_model, lab)
    except: ch = np.nan
    try: db = davies_bouldin_score(X_model, lab)
    except: db = np.nan
    rows.append((k, sil, ch, db))

metrics_df = pd.DataFrame(rows, columns=['k','silhouette','calinski_harabasz','davies_bouldin'])
display(metrics_df)


In [ ]:

# Plots for K selection
import matplotlib.pyplot as plt

plt.figure()
plt.plot(metrics_df['k'], inertias, marker='o')
plt.title("Elbow Plot (Inertia vs k)")
plt.xlabel("k"); plt.ylabel("Inertia")
plt.tight_layout(); plt.show()

plt.figure()
plt.plot(metrics_df['k'], metrics_df['silhouette'], marker='o')
plt.title("Silhouette vs k")
plt.xlabel("k"); plt.ylabel("Silhouette")
plt.tight_layout(); plt.show()

plt.figure()
plt.plot(metrics_df['k'], metrics_df['calinski_harabasz'], marker='o')
plt.title("Calinski–Harabasz vs k")
plt.xlabel("k"); plt.ylabel("CH score")
plt.tight_layout(); plt.show()

plt.figure()
plt.plot(metrics_df['k'], metrics_df['davies_bouldin'], marker='o')
plt.title("Davies–Bouldin vs k (lower better)")
plt.xlabel("k"); plt.ylabel("DB index")
plt.tight_layout(); plt.show()


In [ ]:

# Choose best k (combined ranking)
dfc = metrics_df.copy()
dfc['rank_sil'] = (-dfc['silhouette']).rank(method='min')
dfc['rank_ch']  = (-dfc['calinski_harabasz']).rank(method='min')
dfc['rank_db']  = (dfc['davies_bouldin']).rank(method='min')
dfc['rank_sum'] = dfc[['rank_sil','rank_ch','rank_db']].sum(axis=1)

best_row = dfc.sort_values(['rank_sum','k']).iloc[0]
best_k = int(best_row['k'])
print("Best k:", best_k)
display(dfc.sort_values('rank_sum').head(10))


In [ ]:

# Final KMeans fit
from sklearn.cluster import KMeans, MiniBatchKMeans

if use_minibatch:
    final_km = MiniBatchKMeans(n_clusters=best_k, batch_size=MINIBATCH_BATCH_SIZE, n_init=20, random_state=42)
else:
    final_km = KMeans(n_clusters=best_k, n_init=25, random_state=42)

labels = final_km.fit_predict(X_model)

# Back to original scale centroids
centroids_model = final_km.cluster_centers_
if USE_PCA_FOR_CLUSTER:
    X_scaled_cent = pca.inverse_transform(centroids_model)
else:
    X_scaled_cent = centroids_model

centroids_orig = pd.DataFrame(
    data = (X_scaled_cent * scaler.scale_) + scaler.mean_,
    columns = list(X.columns)
)
centroids_orig.insert(0, 'Cluster', range(best_k))

assignments = pd.DataFrame({'Cluster': labels}, index=X.index).reset_index()
if meta is not None:
    assign_out = pd.concat([meta.reset_index(drop=True), assignments], axis=1)
else:
    assign_out = assignments.copy()

display(assign_out.head(20))
display(centroids_orig.head(20))


In [ ]:

# Profiling & representatives
feat_cols = list(X.columns)
df_lab = pd.DataFrame({'Cluster': labels}, index=X.index)
X_prof = pd.concat([X.reset_index(drop=True), df_lab.reset_index(drop=True)], axis=1)

profile_mean = X_prof.groupby('Cluster')[feat_cols].mean()
profile_std  = X_prof.groupby('Cluster')[feat_cols].std(ddof=0)
display(profile_mean.head(20)); display(profile_std.head(20))

# Representatives (closest to centroid in model space)
import numpy as np
from numpy.linalg import norm

def representative_indices_per_cluster(Xm, labels, centers, topn=10):
    reps = {}
    for c in range(centers.shape[0]):
        idx = np.where(labels == c)[0]
        if len(idx) == 0: 
            reps[c] = []
            continue
        Xm_c = Xm[idx]
        d = norm(Xm_c - centers[c], axis=1)
        order = np.argsort(d)[:topn]
        reps[c] = idx[order]
    return reps

reps = representative_indices_per_cluster(X_model, labels, final_km.cluster_centers_, topn=10)

if meta is not None and (colmap.get('track_name') in meta.columns if meta is not None else False):
    rows = []
    for c, idxs in reps.items():
        for j in idxs:
            tr = meta.iloc[j] if meta is not None else {}
            rows.append((c, tr.get(colmap.get('track_name'), None), tr.get(colmap.get('artist'), None)))
    rep_df = pd.DataFrame(rows, columns=['Cluster','Track','Artist'])
    display(rep_df.head(50))
else:
    print({k: list(v) for k,v in reps.items()})


In [ ]:

# Visualizations
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

pca2 = PCA(n_components=2, random_state=42)
X_p2 = pca2.fit_transform(X_model)

plt.figure(figsize=(7,5))
for c in range(best_k):
    m = (labels == c)
    plt.scatter(X_p2[m,0], X_p2[m,1], s=5, label=f'Cluster {c}')
plt.title(f"K-Means Clusters (PCA 2D) — k={best_k}")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.legend(markerscale=3, loc='best')
plt.tight_layout(); plt.show()

unique, counts = np.unique(labels, return_counts=True)
size_df = pd.DataFrame({'Cluster': unique, 'Count': counts}).sort_values('Count', ascending=False)
display(size_df)

plt.figure()
plt.bar(size_df['Cluster'].astype(str), size_df['Count'].values)
plt.title("Cluster Size Distribution")
plt.xlabel("Cluster"); plt.ylabel("Count")
plt.tight_layout(); plt.show()


In [ ]:

# External validation vs genre
from sklearn.metrics import normalized_mutual_info_score

def purity_score(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    labels = np.unique(y_pred)
    total = len(y_true)
    correct = 0
    for l in labels:
        idx = np.where(y_pred == l)[0]
        if len(idx) == 0: 
            continue
        vals, cnts = np.unique(y_true[idx], return_counts=True)
        correct += cnts.max()
    return correct / total if total > 0 else np.nan

if colmap.get("genre") and (colmap["genre"] in df.columns):
    g = work.loc[X.index, colmap["genre"]].astype(str).values
    purity = purity_score(g, labels)
    nmi = normalized_mutual_info_score(g, labels)
    print("Purity vs genre:", purity)
    print("NMI vs genre:", nmi)
    tmp = pd.DataFrame({'Cluster': labels, 'genre': g})
    topg = tmp.groupby('Cluster')['genre'].agg(lambda x: x.value_counts().head(3).to_dict())
    display(topg)
else:
    print("Kolom genre tidak tersedia — lewati validasi eksternal ini.")


In [ ]:

# Stability bootstrap (ARI)
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans, MiniBatchKMeans

BASE_LABELS = labels
B = 20
aris = []
rng = np.random.default_rng(42)

for b in range(B):
    idx = rng.choice(len(X_model), size=int(0.8*len(X_model)), replace=True)
    Xm = X_model[idx]
    if use_minibatch:
        km_b = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=10, random_state=42+b).fit(Xm)
    else:
        km_b = KMeans(n_clusters=best_k, n_init=10, random_state=42+b).fit(Xm)
    pred_full = km_b.predict(X_model)
    ari = adjusted_rand_score(BASE_LABELS, pred_full)
    aris.append(ari)

print(f"Bootstrap ARI (B={B}): mean={np.mean(aris):.3f}, std={np.std(aris):.3f}, min={np.min(aris):.3f}, max={np.max(aris):.3f}")


In [ ]:

# Ablation: RAW vs PCA10
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

pca_ab = PCA(n_components=min(10, X_scaled.shape[1]), random_state=42)
X_pca_ab = pca_ab.fit_transform(X_scaled)

def fit_scores(Xm, k, minibatch=False):
    if minibatch:
        km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=10, random_state=42)
    else:
        km = KMeans(n_clusters=k, n_init=10, random_state=42)
    lab = km.fit_predict(Xm)
    try: sil = silhouette_score(Xm, lab)
    except: sil = np.nan
    try: ch = calinski_harabasz_score(Xm, lab)
    except: ch = np.nan
    try: db = davies_bouldin_score(Xm, lab)
    except: db = np.nan
    return sil, ch, db

sil_raw, ch_raw, db_raw = fit_scores(X_model, best_k, minibatch=use_minibatch)
sil_pca, ch_pca, db_pca = fit_scores(X_pca_ab, best_k, minibatch=use_minibatch)

print("RAW  -> Sil:", sil_raw, " CH:", ch_raw, " DB:", db_raw)
print("PCA10-> Sil:", sil_pca, " CH:", ch_pca, " DB:", db_pca)


In [ ]:

# Export outputs
out_dir = Path("/mnt/data")
out_dir.mkdir(parents=True, exist_ok=True)

labels_path = out_dir / "kmeans_labels.csv"
centroids_path = out_dir / "kmeans_centroids_original_scale.csv"
profiles_mean_path = out_dir / "kmeans_cluster_profile_mean.csv"
profiles_std_path  = out_dir / "kmeans_cluster_profile_std.csv"
metrics_path = out_dir / "kmeans_metrics_vs_k.csv"

assign_out.to_csv(labels_path, index=False)
centroids_orig.to_csv(centroids_path, index=False)
profile_mean.to_csv(profiles_mean_path)
profile_std.to_csv(profiles_std_path)
metrics_df.to_csv(metrics_path, index=False)

print("Saved:")
print(labels_path)
print(centroids_path)
print(profiles_mean_path)
print(profiles_std_path)
print(metrics_path)



## Kesimpulan (Template)
- **Tujuan**: Mengidentifikasi arketipe mood/typology lagu menggunakan **K-Means** pada fitur audio berskala besar.
- **Pemilihan k**: Gabungan **Silhouette/CH/DB** → k = ___ (kompromi stabil & interpretabel).
- **Hasil**: ___ klaster; karakter ringkas per klaster; contoh lagu/artis representatif.
- **Validasi**: Purity = ___; NMI = ___ terhadap **genre** (jika tersedia).
- **Stabilitas**: Bootstrap ARI = mean ___ ± ___ (___).
- **Ablasi**: Perbandingan RAW vs PCA10 menunjukkan ___; konfigurasi final menggunakan ___.
- **Keterbatasan**: Sensitivitas terhadap skala & korelasi fitur; asumsi bentuk cluster sferis.
- **Kerja lanjut**: Micro-cluster per genre, trend per dekade, bandingkan dengan HDBSCAN atau spherical K-Means.
